## 1. 데이터 불러오기

In [46]:
import pandas as pd

# 학습 데이터 로딩
df17 = pd.read_csv('./kaggle_data/test2017.csv', index_col=0)
df18 = pd.read_csv('./kaggle_data/test2018.csv', index_col=0)
df19 = pd.read_csv('./kaggle_data/test2019.csv', index_col=0)
df20 = pd.read_csv('./kaggle_data/test2020.csv', index_col=0)
df21 = pd.read_csv('./kaggle_data/test2021.csv', index_col=0)

In [47]:
df17.head()

,기준일ID,시간대구분,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,남자35세부터39세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20170101,0,43922.2281,953.7652,627.7225,927.6134,2025.0630,2278.7336,1744.8161,1713.6754,...,2060.1511,1734.3805,1713.3332,1754.6515,1636.4175,1521.3668,1728.0639,1484.4518,1238.2606,3670.0451
1,20170101,1,43763.4929,962.2909,633.3292,970.6215,2045.7002,2412.6287,1765.7725,1765.1171,...,2037.5677,1745.5960,1735.4797,1700.9196,1619.3672,1474.8418,1661.3750,1470.3009,1098.4788,3568.8258
2,20170101,2,43561.5708,932.2939,613.6027,964.3505,2057.3601,2419.0941,1734.0569,1746.3940,...,2005.4838,1750.7594,1710.6116,1660.6697,1660.9852,1511.4103,1635.0352,1441.0173,1112.3632,3541.2620
3,20170101,3,43928.9929,926.3617,609.7009,983.2400,2100.5071,2480.1898,1780.3833,1723.6640,...,2044.6913,1777.4169,1718.1475,1678.9374,1626.8350,1516.5837,1665.8781,1471.2127,1098.3154,3630.0822
4,20170101,4,44150.2973,922.8893,607.4184,990.0534,2128.2729,2531.6549,1774.7853,1752.0073,...,2060.8824,1801.2144,1718.2957,1678.6174,1640.9056,1513.5035,1653.2704,1435.5180,1090.4059,3569.1653


In [48]:
df_total = pd.concat([df17, df18, df19, df20, df21], axis=0)

---

## 2. 데이터 전처리

In [49]:
# 열 추가
df_total['기준일ID'] = pd.to_datetime(df_total['기준일ID'], format='%Y%m%d')

df_total['year']=df_total['기준일ID'].dt.year
df_total['month']=df_total['기준일ID'].dt.month
df_total['day']=df_total['기준일ID'].dt.day

In [50]:
# 열 제거 및 이름 변경
df_total.rename(columns={'시간대구분': 'hour'}, inplace=True)
df_total = df_total.loc[:, ['year', 'month', 'day', 'hour', '총생활인구수']].copy()

In [51]:
# 시프트를 사용해 target 설정
df_total['총생활인구수_M2'] = df_total['총생활인구수'].shift(-1416) # 2개월 후 동일 시간대의 총 생활인구수

In [52]:
# 롤링을 사용해 feature 추가
df_total['총생활인구수_MA_1W'] = df_total['총생활인구수'].rolling(168).mean() # 기준일 포함 최근 7일간 이동 평균
df_total['총생활인구수_MA_2W'] = df_total['총생활인구수'].rolling(336).mean() # 기준일 포함 최근 14일간 이동 평균
df_total['총생활인구수_MA_1M'] = df_total['총생활인구수'].rolling(720).mean() # 기준일 포함 최근 30일간 이동 평균
df_total['총생활인구수_MA_2M'] = df_total['총생활인구수'].rolling(1440).mean() # 기준일 포함 최근 60일간 이동 평균

In [53]:
df_total.tail()

,year,month,day,hour,총생활인구수,총생활인구수_M2,총생활인구수_MA_1W,총생활인구수_MA_2W,총생활인구수_MA_1M,총생활인구수_MA_2M
8755,2021,12,31,19,33304.5400,NaN,36873.241666,37250.152718,37691.374041,37657.979762
8756,2021,12,31,20,33368.7201,NaN,36858.370712,37239.861731,37687.598821,37655.916790
8757,2021,12,31,21,34083.1721,NaN,36845.175405,37232.869284,37684.310276,37654.219290
8758,2021,12,31,22,34380.0806,NaN,36833.093603,37225.700260,37680.570921,37652.441130
8759,2021,12,31,23,37074.2785,NaN,36820.045605,37218.365621,37676.783771,37650.091112


In [54]:
# 불필요한 행 제거
cols = ['year', 'month', 'day', 'hour']

df_total.drop(cols, axis=1, inplace=True)
df_total.reset_index(drop=True, inplace=True)

In [55]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df_total, test_size=1416, shuffle=False)

In [56]:
# x, y 분리
train_set.dropna(axis=0, inplace=True)
train_x = train_set.drop('총생활인구수_M2', axis=1)
train_y = train_set['총생활인구수_M2']

test_x = test_set.drop('총생활인구수_M2', axis=1)

----

## 3. 모델링

In [57]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import *

In [58]:
scaler = MinMaxScaler()
train_x_s = scaler.fit_transform(train_x)
test_x_s = scaler.transform(test_x)

### 가. Random Forest

In [59]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(train_x_s, train_y.ravel())

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [60]:
pred_rf = rf.predict(test_x_s)

pred_rf = pd.DataFrame({'count': pred_rf})
pred_rf.to_csv('./kaggle_data/rf_add4features.csv', index_label='id')

### 나. XGB

In [61]:
from xgboost import XGBRegressor

xgb = XGBRegressor()
xgb.fit(train_x_s, train_y.ravel())

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=0, reg_alpha=0, ...)

In [62]:
pred_xgb = xgb.predict(test_x_s)

pred_xgb = pd.DataFrame({'count': pred_xgb})
pred_xgb.to_csv('./kaggle_data/xgb_add4features.csv', index_label='id')

### 다. LightGBM

In [63]:
import lightgbm as lgb

lgbm = lgb.LGBMRegressor()
lgbm.fit(train_x_s, train_y.ravel())

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [64]:
pred_lgbm = lgbm.predict(test_x_s)

pred_lgbm = pd.DataFrame({'count': pred_lgbm})
pred_lgbm.to_csv('./kaggle_data/lgbm_add4features.csv', index_label='id')

### 라. CatBoost

In [65]:
import catboost

cb = catboost.CatBoostRegressor()
cb.fit(train_x_s, train_y.ravel())

Learning rate set to 0.073523
0:	learn: 3557.7540183	total: 5.09ms	remaining: 5.08s
1:	learn: 3431.2677345	total: 10.1ms	remaining: 5.01s
2:	learn: 3316.0672194	total: 14.2ms	remaining: 4.72s
3:	learn: 3212.0042220	total: 18.3ms	remaining: 4.55s
4:	learn: 3118.2964996	total: 23.6ms	remaining: 4.7s
5:	learn: 3033.0887091	total: 28.1ms	remaining: 4.65s
6:	learn: 2957.4955005	total: 32.5ms	remaining: 4.62s
7:	learn: 2889.2788891	total: 37.2ms	remaining: 4.61s
8:	learn: 2832.3446350	total: 41.5ms	remaining: 4.57s
9:	learn: 2778.1993318	total: 46.8ms	remaining: 4.63s
10:	learn: 2729.9072668	total: 51.4ms	remaining: 4.62s
11:	learn: 2689.3191994	total: 56.2ms	remaining: 4.63s
12:	learn: 2652.0548953	total: 60.9ms	remaining: 4.63s
13:	learn: 2615.5450883	total: 65.8ms	remaining: 4.63s
14:	learn: 2587.6110913	total: 69.8ms	remaining: 4.58s
15:	learn: 2558.6147087	total: 74.3ms	remaining: 4.57s
16:	learn: 2533.5344993	total: 79ms	remaining: 4.57s
17:	learn: 2509.9698042	total: 82.9ms	remaining:

In [66]:
pred_cb = cb.predict(test_x_s)

pred_cb = pd.DataFrame({'count': pred_cb})
pred_cb.to_csv('./kaggle_data/cb_add4features.csv', index_label='id')

### 마. PyCaret

In [12]:
from pycaret.regression import *

In [67]:
reg = setup(train_set, 
            preprocess = False, # True로 설정 시 자체적인 Feature Engineering을 추가로 진행해 Predict 불가능
            train_size = 0.999,
            target = '총생활인구수_M2', # 목표 변수
            silent = True,
            use_gpu = True, # CatBoost 속도 향상
            )

,Description,Value
0,session_id,3450
1,Target,총생활인구수_M2
2,Original Data,"(40657, 6)"
3,Missing Values,False
4,Numeric Features,5
5,Categorical Features,0
6,Transformed Train Set,"(40616, 5)"
7,Transformed Test Set,"(41, 5)"
8,Shuffle Train-Test,True
9,Stratify Train-Test,False


In [68]:
top3 = compare_models(n_select = 3, sort = 'RMSE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,735.9506,1274075.6019,1128.5821,0.9069,0.0279,0.0183,16.9120
rf,Random Forest Regressor,819.4603,1525292.9382,1234.7000,0.8886,0.0306,0.0204,14.6850
xgboost,Extreme Gradient Boosting,1021.4574,1808031.0000,1344.5022,0.8679,0.0333,0.0253,0.7180
catboost,CatBoost Regressor,1105.3770,2058399.9720,1434.6008,0.8496,0.0353,0.0273,4.9000
lightgbm,Light Gradient Boosting Machine,1262.0451,2760403.7970,1661.2597,0.7984,0.0410,0.0312,1.1150
dt,Decision Tree Regressor,1023.9221,2891857.1204,1699.6753,0.7886,0.0419,0.0254,0.2730
knn,K Neighbors Regressor,1208.3587,3031055.2500,1740.4974,0.7787,0.0433,0.0301,0.7450
gbr,Gradient Boosting Regressor,1562.4958,4454480.1088,2110.2139,0.6746,0.0520,0.0386,5.6000
br,Bayesian Ridge,1894.6122,6567408.3896,2562.5055,0.5204,0.0632,0.0468,0.0150
lar,Least Angle Regression,1894.6122,6567410.0566,2562.5058,0.5204,0.0632,0.0468,0.0130


In [ ]:
models = []
for m in top3:
    models.append(tune_model(m, optimize = 'RMSE', choose_better = True, n_iter = 10))

In [ ]:
xgb = models[0]
pred_xgb = xgb.predict(test_x)
pred_xgb = pd.DataFrame({'count': pred_xgb})
pred_xgb.to_csv('./kaggle_data/xgb_optimized.csv', index_label='id')

In [23]:
cb = models[1]
pred_cb = cb.predict(test_x)
pred_cb = pd.DataFrame({'count': pred_cb})
pred_cb.to_csv('./kaggle_data/cb_optimized.csv', index_label='id')

In [24]:
lgbm = models[2]
pred_lgbm = lgbm.predict(test_x)
pred_lgbm = pd.DataFrame({'count': pred_lgbm})
pred_lgbm.to_csv('./kaggle_data/lgbm_optimized.csv', index_label='id')

In [20]:
voting = blend_models(models, optimize = 'RMSE')
voting = tune_model(voting, optimize = 'RMSE', choose_better = True, n_iter = 30)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1669.5848,5192568.4194,2278.7208,0.6194,0.0560,0.0412
1,1713.0040,5607805.3132,2368.0805,0.5933,0.0594,0.0426
2,1680.8913,5240179.2285,2289.1438,0.6200,0.0568,0.0417
3,1671.6584,5276353.6997,2297.0315,0.6087,0.0564,0.0412
4,1657.9648,5109589.8065,2260.4402,0.6228,0.0560,0.0410
5,1697.2620,5266987.3268,2294.9918,0.6052,0.0566,0.0419
6,1644.3444,4980438.1668,2231.6895,0.6247,0.0548,0.0405
7,1690.1197,5265141.6059,2294.5896,0.5994,0.0568,0.0417
8,1688.6825,5425860.5856,2329.3477,0.6064,0.0577,0.0419


In [21]:
voting = finalize_model(voting)
pred = voting.predict(test_x)
pred = pd.DataFrame({'count': pred})
pred.to_csv('./kaggle_data/pycaret.csv', index_label='id')